In [1]:
import numpy as np
import h5py

from matplotlib.pylab import *
import matplotlib.pyplot as plt
import numpy.linalg as la
import bsk

import pandas as pd
import os
from tqdm.notebook import tqdm

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 25,
         'axes.titlesize': 25,
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
mpl.rcParams.update(params)
plt.jet()

path = './data/'

<Figure size 1080x360 with 0 Axes>

In [ ]:
#----------------- Reading assembly info
info = np.loadtxt('/Users/usuario-mac/Desktop/NEW_301/Galaxy/Galaxies_All/Data_Skeleton/Quartiles/edadesLimpias.txt')

IDH = np.int_(info[:,0])
IDG = np.int_(info[:,1])
z_formation = info[:,2]
age_quartil = np.int_(info[:,3]) #-1(no tree),1(1Q),2(2Q),3(3Q),4(4Q)
galaxy_mass = info[:,4]

pos_file = "Pos_Limpias.txt"

#----------------- Computing beta-skeleton with Filipo code
if os.path.isfile('./HackingLSSCode/bin/LSS_BSK_calc'):
    print("Computing BSK for Beta=1.0")
    command = 'sh run_BSK.sh '+path+pos_file+' '+pos_file[:-4]+' 1.0'
    print (command)
    os.system(command)
    print("BSK completed! \n "+pos_file[:-4]+".BSKIndex created!")

    #----------------- Reading files with beta-skeleton results
    betafile = pos_file[:-4]+".BSKIndex"
    ca, cb = loadtxt(path+betafile, unpack = True)
    x,y,z = loadtxt(path+pos_file, unpack = True)
    ca = ca.astype(int)
    cb = cb.astype(int)

    #----------------- Computing all features form b-skeleton
    Max = len(x)
    ID,nc,con = bsk.number_connections(Max,ca,cb)

    ad,vol,den = bsk.features(Max,ID,x,y,z,con,nc)

    #----------------- New definitons of ad,nc, den
    ad = ad/np.mean(ad)    
    ncn = nc.astype(float)-np.median(nc.astype(float))
    den = np.log(den)

    Dnc, Dad, Dden = bsk.neigh_features(Max,ID,nc,ncn,ad,den,con)
else:
    print("Error: First run the makefile in HackingLSSCode/src/")

In [ ]:
zin =3
zlim = (max(z) - min(z))/2 + min(z)# Plane in z axis to plot the Bskeleton
maskzm = z < (zlim + zin)
maskzM = z > (zlim - zin)
IDzm = ID[maskzm & maskzM]

fig = plt.figure(figsize = (10,10))
plt.title(r'$\beta$-Skeleton',size=30)
plt.xlabel("x [Mpc]",size=25)
plt.ylabel("y [Mpc]",size=25)
for k in IDzm:
    for j in con[k]:
        plt.plot([x[k],x[j]],[y[k],y[j]],'bo-',markersize=2, c="#0a337a")
plt.savefig('./figures/bskQ1.png', bbox_inches='tight', resterized=True, transparent=True)
plt.show()

In [ ]:
from sklearn.ensemble.forest import RandomForestClassifier
import pickle
classes = np.array(['Peaks','Filaments','Sheets','Voids'])

In [ ]:
Xdata = pd.DataFrame({'nc':nc,'ad':ad,'den':den,'Dnc':Dnc,'Dad':Dad,'Dden':Dden})
clf = pickle.load(open(path+'cosmicweb_bsk_model_mass.sav','rb'))
environment = clf.predict(Xdata)

In [ ]:
f1 = h5py.File(path+'all_features.hdf5','w')
dt = h5py.special_dtype(vlen=np.dtype('int64'))
f1.create_dataset('ID', data = ID)
f1.create_dataset('nc', data = nc)
f1.create_dataset('coor', data = (x,y,z))
f1.create_dataset('ad', data = ad)
f1.create_dataset('den', data = den)
f1.create_dataset('Dnc', data = Dnc)
f1.create_dataset('Dad', data = Dad)
f1.create_dataset('Dden', data = Dden)
f1.create_dataset('IDH', data = IDH)#ID del halo al que pertenece cada galaxia
f1.create_dataset('IDG', data = IDG)#ID real de las galaxias
f1.create_dataset('z',data = z_formation)
f1.create_dataset('age', data = age_quartil)
f1.create_dataset('galaxy_mass', data = galaxy_mass)
f1.create_dataset('enviroment', data = environment)
f1.close()